In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

/home/kk/dl/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('/home/kk/Data/mnist',one_hot=True)

Extracting /home/kk/Data/mnist/train-images-idx3-ubyte.gz
Extracting /home/kk/Data/mnist/train-labels-idx1-ubyte.gz
Extracting /home/kk/Data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/kk/Data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.train.images.shape

(55000, 784)

In [4]:
mnist.train.labels.shape

(55000, 10)

In [5]:
learning_rate = 0.001
training_iters = 100000
batch_size =256
display_step = 10

In [15]:
n_input = 28
n_steps = 28 #timestep
n_hidden = 128#hidden layers
n_classes =10

In [7]:
x=tf.placeholder ('float',[None,n_steps,n_input],name='X')
y=tf.placeholder ('float',[None,n_classes],name='Y')


In [8]:
weights ={'out':tf.Variable(tf.random_normal([n_hidden,n_classes]),name='Weight')}
biases ={'out':tf.Variable(tf.random_normal([n_classes]),name='bias')}

In [9]:
def RNN(x,weights,bias):
    x=tf.unstack(x,n_steps,1)
    lstm_cell=rnn.BasicLSTMCell(n_hidden,forget_bias=0.8)
    outputs, states =rnn.static_rnn(lstm_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],weights['out']+biases['out'])

In [10]:
pred = RNN(x,weights,biases)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y),name=None)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [13]:
init = tf.global_variables_initializer()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./lstm_mnist',sess.graph)
    step = 1
    while step*batch_size < training_iters:
        batch_x,batch_y =mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape(batch_size,n_steps,n_input)
        sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
        if step % display_step ==0:
            acc = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            loss = sess.run(cost,feed_dict={x:batch_x,y:batch_y})
            print('Iter:'+str(step*batch_size)+', Minibatch Loss='+\
             '{:.6f}'.format(loss)+', Trainging Accurary='+\
             '{:.5f}'.format(acc))
        step+=1
    print('Optimization Finished!')
    writer.close()
    test_len =128
    test_data = mnist.test.images[:test_len].reshape((-1,n_steps,n_input))
    test_label = mnist.test.labels[:test_len]
    print('Test Accuracy:', \
          sess.run(accuracy,feed_dict={x:test_data,y:test_label}))

Iter:2560, Minibatch Loss=1.863115, Trainging Accurary=0.37109
Iter:5120, Minibatch Loss=1.403156, Trainging Accurary=0.51562
Iter:7680, Minibatch Loss=1.129744, Trainging Accurary=0.67578
Iter:10240, Minibatch Loss=1.054204, Trainging Accurary=0.66406
Iter:12800, Minibatch Loss=0.737983, Trainging Accurary=0.77734
Iter:15360, Minibatch Loss=0.684760, Trainging Accurary=0.78516
Iter:17920, Minibatch Loss=0.549214, Trainging Accurary=0.78516
Iter:20480, Minibatch Loss=0.472337, Trainging Accurary=0.85156
Iter:23040, Minibatch Loss=0.387113, Trainging Accurary=0.85938
Iter:25600, Minibatch Loss=0.412277, Trainging Accurary=0.85938
Iter:28160, Minibatch Loss=0.320474, Trainging Accurary=0.89062
Iter:30720, Minibatch Loss=0.330570, Trainging Accurary=0.89453
Iter:33280, Minibatch Loss=0.333233, Trainging Accurary=0.89844
Iter:35840, Minibatch Loss=0.238042, Trainging Accurary=0.92969
Iter:38400, Minibatch Loss=0.244601, Trainging Accurary=0.92188
Iter:40960, Minibatch Loss=0.230201, Traing